In [4]:
from __future__ import annotations

import os
import logging
from typing_extensions import Literal, Any

# These spam a bunch of warnings/errors about modded content. Comment out if you need to debug.
for l in [
    "PyMCTranslate.py3.api.version.translators.base",
    "PyMCTranslate.py3.api.version.translators.biome",
    "amulet.api.wrapper.format_wrapper",
]:
    logging.getLogger(l).disabled = True

import amulet

In [5]:
class MinecraftWorld:
    def __init__(self, path: os.PathLike) -> None:
        self.data = amulet.load_level(path)
        print(f'World {os.path.split(path[-1])} successfully loaded.')

    def _safe_load_chunk(self, xchunk: int, ychunk: int, in_dim: Literal['minecraft:the_end', 'minecraft:nether', 'minecraft:overworld'] | Any) -> amulet.api.chunk.Chunk:
        try:
            return self.data.get_chunk(xchunk, ychunk, in_dim)
        except amulet.api.errors.ChunkLoadError:
            pass

    def find_blocks(self, blocks: list[str], in_dim: Literal['minecraft:the_end', 'minecraft:nether', 'minecraft:overworld'] | Any) -> list[tuple[int, int, int]]:
        # get chunk coords and load
        chunk_coords = list(self.data.chunks.all_chunk_coords(in_dim))
        chunks = [self._safe_load_chunk(x, y, in_dim) for x, y in chunk_coords]

In [6]:
world = MinecraftWorld(r'C:\Users\Huan Zheng\curseforge\minecraft\Instances\Not Too Complicated 2\saves\New World')
world.find_blocks(blocks=["emendatusenigmatica:osmium_end_stone_ore"], in_dim='minecraft:the_end')

INFO - Loading level C:\Users\Huan Zheng\curseforge\minecraft\Instances\Not Too Complicated 2\saves\New World


In [1]:
# Requires: pip install amulet-core

import logging
import sys
from typing import List, Tuple, TypeVar

import amulet

T = TypeVar("T")
WORLD_PATH = r'C:\Users\Huan Zheng\curseforge\minecraft\Instances\Not Too Complicated 2\saves\New World'
DESIRED_BLOCKS = ["emendatusenigmatica:osmium_end_stone_ore"]
TARGET_DIMENSION = "minecraft:the_end"


def chunk_list(list: List[T], interval=10) -> List[List[T]]:
    """Splits a list into a list of lists, each containing `interval` items"""
    return [list[i : i + interval] for i in range(0, len(list), interval)]


def safe_load_chunk(
    world: amulet.api.level.World,
    dim: amulet.api.data_types.Dimension,
    cx: int,
    cy: int,
):
    """Attempts to load a chunk from the world, and returns None if it fails"""
    try:
        return world.get_chunk(cx, cy, dim)
    except amulet.api.errors.ChunkLoadError:
        pass


def safe_load_chunks(
    world: amulet.api.level.World,
    dim: amulet.api.data_types.Dimension,
    coords: List[Tuple[int, int]],
):
    """Safely loads the chunks with the specified coordinates"""
    chunks = [safe_load_chunk(world, dim, cx, cy) for cx, cy in coords]
    successful_chunks = [c for c in chunks if c is not None]
    return (successful_chunks, len(coords), len(successful_chunks))


def get_non_air_blocks(chunk: amulet.api.chunk.Chunk, height=256):
    """Returns the position & block reference for each position inside a chunk"""
    return (
        ((chunk.cx * 16 + x, y, chunk.cz * 16 + z), chunk.get_block(x, y, z))
        for x in range(0, 16)
        for y in range(0, height)
        for z in range(0, 16)
        if chunk.get_block(x, y, z).namespaced_name != "universal_minecraft:air"
    )


def search_for_blocks(
    world: amulet.api.level.World,
    dimension: amulet.api.data_types.Dimension,
    target_blocks: List[str],
):
    coords = list(world.chunks.all_chunk_coords(dimension))
    total_attempted = 0
    total_succeeded = 0

    for coords_batch in chunk_list(coords, 16):
        (chunks, attempted, succeeded) = safe_load_chunks(
            world, dimension, coords_batch
        )
        total_attempted += attempted

        blocks = [(p, b) for c in chunks for (p, b) in get_non_air_blocks(c)]
        found_blocks = [
            (p, b) for (p, b) in blocks if b.namespaced_name in target_blocks
        ]
        if len(found_blocks) != 0:
            print("Found {} blocks".format(len(found_blocks)))
            print(
                "\n".join(
                    ["{} - {}".format(b.namespaced_name, p) for (p, b) in found_blocks]
                )
            )

        total_succeeded += succeeded
        print(
            "Loaded batch with {}/{} chunks ({}/{})".format(
                succeeded,
                attempted,
                total_succeeded,
                total_attempted,
            ),
            file=sys.stderr,
        )

    print(
        "Finished with {}/{} chunks".format(total_succeeded, total_attempted),
        file=sys.stderr,
    )


# These spam a bunch of warnings/errors about modded content. Comment out if you need to debug.
for l in [
    "PyMCTranslate.py3.api.version.translators.base",
    "PyMCTranslate.py3.api.version.translators.biome",
    "amulet.api.wrapper.format_wrapper",
]:
    logging.getLogger(l).disabled = True

search_for_blocks(
    amulet.load_level(WORLD_PATH),
    TARGET_DIMENSION,
    DESIRED_BLOCKS,
)

INFO - PyMCTranslate Version 261
INFO - Loading level C:\Users\Huan Zheng\curseforge\minecraft\Instances\Not Too Complicated 2\saves\New World
Loaded batch with 16/16 chunks (16/16)
Loaded batch with 16/16 chunks (32/32)
Loaded batch with 16/16 chunks (48/48)
Loaded batch with 16/16 chunks (64/64)
Loaded batch with 16/16 chunks (80/80)
Loaded batch with 16/16 chunks (96/96)
Loaded batch with 16/16 chunks (112/112)
Loaded batch with 16/16 chunks (128/128)
Loaded batch with 16/16 chunks (144/144)
Loaded batch with 16/16 chunks (160/160)
Loaded batch with 14/16 chunks (174/176)
Loaded batch with 16/16 chunks (190/192)
Loaded batch with 16/16 chunks (206/208)
Loaded batch with 16/16 chunks (222/224)
Loaded batch with 16/16 chunks (238/240)
Loaded batch with 16/16 chunks (254/256)
Loaded batch with 16/16 chunks (270/272)
Loaded batch with 16/16 chunks (286/288)
Loaded batch with 16/16 chunks (302/304)
Loaded batch with 16/16 chunks (318/320)
Loaded batch with 16/16 chunks (334/336)
Loaded b

Found 433 blocks
emendatusenigmatica:osmium_end_stone_ore - (1456, 58, 1473)
emendatusenigmatica:osmium_end_stone_ore - (1456, 58, 1474)
emendatusenigmatica:osmium_end_stone_ore - (1456, 60, 1474)
emendatusenigmatica:osmium_end_stone_ore - (1456, 60, 1475)
emendatusenigmatica:osmium_end_stone_ore - (1456, 61, 1475)
emendatusenigmatica:osmium_end_stone_ore - (1456, 61, 1476)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1472)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1473)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1474)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1475)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1476)
emendatusenigmatica:osmium_end_stone_ore - (1457, 58, 1477)
emendatusenigmatica:osmium_end_stone_ore - (1457, 59, 1472)
emendatusenigmatica:osmium_end_stone_ore - (1457, 59, 1473)
emendatusenigmatica:osmium_end_stone_ore - (1457, 59, 1474)
emendatusenigmatica:osmium_end_stone_ore - (1457, 59, 1475)
emendatusenigmatica:osm

Loaded batch with 15/16 chunks (810/816)
Loaded batch with 15/16 chunks (825/832)
Loaded batch with 15/16 chunks (840/848)
Loaded batch with 16/16 chunks (856/864)
Loaded batch with 16/16 chunks (872/880)
Loaded batch with 16/16 chunks (888/896)
Loaded batch with 16/16 chunks (904/912)
Loaded batch with 16/16 chunks (920/928)
Loaded batch with 16/16 chunks (936/944)
Loaded batch with 16/16 chunks (952/960)
Loaded batch with 16/16 chunks (968/976)
Loaded batch with 16/16 chunks (984/992)
Loaded batch with 16/16 chunks (1000/1008)


Found 631 blocks
emendatusenigmatica:osmium_end_stone_ore - (-1184, 32, 40)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 32, 41)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 33, 40)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 33, 41)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 36, 36)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 36, 37)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 36, 38)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 36, 39)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 37, 36)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 37, 37)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 37, 38)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 38, 37)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 38, 38)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 39, 38)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 39, 39)
emendatusenigmatica:osmium_end_stone_ore - (-1184, 39, 40)
emendatusenigmatica:osmium_end_stone_or

Loaded batch with 16/16 chunks (1016/1024)
Loaded batch with 16/16 chunks (1032/1040)
Loaded batch with 16/16 chunks (1048/1056)
Loaded batch with 16/16 chunks (1064/1072)
Loaded batch with 16/16 chunks (1080/1088)
Loaded batch with 16/16 chunks (1096/1104)
Loaded batch with 16/16 chunks (1112/1120)
Loaded batch with 16/16 chunks (1128/1136)
Loaded batch with 16/16 chunks (1144/1152)
Loaded batch with 15/16 chunks (1159/1168)
Loaded batch with 16/16 chunks (1175/1184)
Loaded batch with 16/16 chunks (1191/1200)
Loaded batch with 16/16 chunks (1207/1216)
Loaded batch with 16/16 chunks (1223/1232)
Loaded batch with 16/16 chunks (1239/1248)
Loaded batch with 16/16 chunks (1255/1264)
Loaded batch with 16/16 chunks (1271/1280)
Loaded batch with 16/16 chunks (1287/1296)
Loaded batch with 16/16 chunks (1303/1312)
Loaded batch with 16/16 chunks (1319/1328)
Loaded batch with 16/16 chunks (1335/1344)
Loaded batch with 16/16 chunks (1351/1360)
Loaded batch with 16/16 chunks (1367/1376)
Loaded batc

KeyboardInterrupt: 